In [2]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.7/648.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.2 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import LabelEncoder


# Load the Excel file

In [14]:
excel_file = 'sms_data.xlsx'
raw_data = pd.read_excel(excel_file, header=None, names=['label', 'message'])

# Save it as a CSV file

In [15]:
raw_data.to_csv('raw_data.csv', index=False)
X_text = raw_data['message']
y = raw_data['label']

# Convert Text to Numerical Features using TF-IDF

In [16]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X = vectorizer.fit_transform(X_text.astype(str))

# Convert labels to numerical using LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

# Split Data

In [17]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set MLflow Tracking URI (Local Directory)

In [18]:

mlflow.set_tracking_uri("file:///content/mlruns")
mlflow.set_experiment("SMS_Spam_Detection")


<Experiment: artifact_location='file:///content/mlruns/727595653916590589', creation_time=1741138284855, experiment_id='727595653916590589', last_update_time=1741138284855, lifecycle_stage='active', name='SMS_Spam_Detection', tags={}>

# Define Models

In [19]:

models = {
    "LogisticRegression": LogisticRegression(),
    "RandomForest": RandomForestClassifier(n_estimators=100),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

aucpr_scores = {}
run_ids = {}

for name, model in models.items():
    print(f"\n Running {name} Model...")
    with mlflow.start_run(run_name=name) as run:
        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1]

        # Compute AUCPR
        aucpr = average_precision_score(y_test, y_pred, pos_label=1)
        aucpr_scores[name] = aucpr

        # Infer model signature
        signature = infer_signature(X_train[:5].toarray(), model.predict_proba(X_train[:5])[:, 1])

        # Log Model Parameters and Metrics
        mlflow.log_param("model_type", name)
        mlflow.log_metric("AUCPR", aucpr)

        # Log Model with Signature
        input_example = X_train[:1].toarray()
        mlflow.sklearn.log_model(model, name, signature=signature, input_example=input_example)
        run_ids[name] = run.info.run_id

        print(f" {name} AUCPR: {aucpr:.4f}")
        print(f" Run ID: {run.info.run_id}")
        print(f" Tracking URL: {mlflow.get_tracking_uri()}/0/{run.info.experiment_id}/{run.info.run_id}")



 Running LogisticRegression Model...
 LogisticRegression AUCPR: 0.9761
 Run ID: a2c7479dad6f4b21ae0e3d411a127833
 Tracking URL: file:///content/mlruns/0/727595653916590589/a2c7479dad6f4b21ae0e3d411a127833

 Running RandomForest Model...
 RandomForest AUCPR: 0.9826
 Run ID: 32ebfb544c5549f5bd2b9a01b7318129
 Tracking URL: file:///content/mlruns/0/727595653916590589/32ebfb544c5549f5bd2b9a01b7318129

 Running XGBoost Model...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:43:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


 XGBoost AUCPR: 0.9602
 Run ID: d41a83932d0048ef94ddf45c2afb036b
 Tracking URL: file:///content/mlruns/0/727595653916590589/d41a83932d0048ef94ddf45c2afb036b


# Print Final AUCPR Summary

In [20]:

print("\n **Final AUCPR Scores:**")
for model_name, score in aucpr_scores.items():
    print(f"{model_name}: AUCPR = {score:.4f}")


 **Final AUCPR Scores:**
LogisticRegression: AUCPR = 0.9761
RandomForest: AUCPR = 0.9826
XGBoost: AUCPR = 0.9602
